In [1]:
import Graph_Amazon_Movies as gam
import Amazon_Movie_Parser as amp
import numpy as np
import pandas as pd
from networkx.algorithms import bipartite
import networkx as nx
import Amazon_Movie_Parser as prs
import datetime
from numpy import savetxt

In [2]:
try:
    # needed only if you don't have the movies.txt file in your local or colab directory
    from google_drive_downloader import GoogleDriveDownloader as gdd
    file_id1='1iytA1n2z4go3uVCwE__vIKouTKyIDjEq'
    # movies.txt >> 9 GB data  ... below shared URL has the fileID to download
    #https://drive.google.com/file/d/1cRLjG6Pl4JEag-utmD-7nLipzxCdmkiQ/view?usp=sharing
    file_id2='1cRLjG6Pl4JEag-utmD-7nLipzxCdmkiQ'
    gdd.download_file_from_google_drive(file_id=file_id2,
                                        dest_path='data/movies.txt',
                                        unzip=True)
except:
    print('no need for laptop')

no need for laptop


In [3]:
start_time = datetime.datetime.now()
grp = gam.Graph_Amazon()
max_connected_gr_amazon_movies = grp.Create_Bipartite(file_name = 'data/movies.txt', \
                                                      n_movies = 10000, prs_out = 'dictionary')
end_time = datetime.datetime.now()
print('Calculation time: {}'.format(end_time-start_time))

Calculation time: 0:00:01.910700


In [4]:
bottom_nodes, top_nodes = bipartite.sets(max_connected_gr_amazon_movies)

In [5]:
# Data check
# some MOVIES don't start with B but instead full numbers like: 0790747324
print('max_connected_gr_amazon_movies:')
count_B = 0
for movie in top_nodes:
    if movie.startswith('B'):
        count_B += 1

print('Movies: {} ... starts with B: {}'.format(len(top_nodes), count_B))

count_A=0
for movie in bottom_nodes:
    if movie.startswith('A'):
        count_A += 1

print('Users: {} ... starts with A: {}'.format(len(bottom_nodes), count_A))

grp.Is_Connected_Bipartite(max_connected_gr_amazon_movies)
# edges between movies
#max_grp_top = bipartite.projected_graph(max_connected_gr_amazon_movies, grp.top_nodes)
# shows all the records BEFORE picking up the MAX CONNECTED component - most likely it will be a DISCONNECTED GRAPH
# returns a LIST
#edge_list_ALL = grp.Get_All_Edges_with_Weight(grp.gr_amazon_movies)
#print(edge_list_ALL[0:5])
#edge_list_connected = grp.Get_All_Edges_with_Weight(max_connected_gr_amazon_movies)
#list(max_connected_gr_amazon_movies.edges)[0:5]
# shows the nodes of the max connected component/subgraph
#grp.Show_Nodes()
#if your graph is too BIG, it is better not to draw it, it takes time..
#grp.Draw_Bipartite(max_connected_gr_amazon_movies)
#shows the edges with weights
#list(max_connected_gr_amazon_movies.edges.data('weight', default=1))[0:5]

max_connected_gr_amazon_movies:
Movies: 167 ... starts with B: 134
Users: 8183 ... starts with A: 8183
Is connected: True ... Is bipartite: True


In [6]:
#saves the movies/graph in the format of:
'''
A141HP4LYPW,B003AI2VGA,3.0
A141HP4LYPW,B000063W1R,4.0
A141HP4LYPW,6304286961,5.0
'''
#max_connected_gr_amazon_movies
file_name_saved = amp.Save_Movies(max_connected_gr_amazon_movies)

Graph info saved in: Movies_Connected_9253_12.5.2020_14.3.1.txt


In [7]:
# returns a list in the format of:
# [['userId', 'productId', 'score'], ['A141HP4LYPW', 'B003AI2VGA', '3.0']]
#the_giant_component_read = amp.Read_Connected_Movies("Movies_Connected_4643_11.5.2020_21.11.17.txt")
the_giant_component_read = amp.Read_Connected_Movies(file_name_saved)
#test[0].split(sep=",")

In [8]:
# it is mostly for a double check to see if the movies were saved to the file properly 
#and check if we read them and create a graph it will still be a connected graph or not
grp_giant = grp.Create_Graph_From_List_WITH_Weight(the_giant_component_read)

In [9]:
# double check if the numbers are matching before saving the records
# and after creating the graph from the saved file
print('BEFORE: # of nodes: {} ... # of edges: {}'.format(len(max_connected_gr_amazon_movies.nodes), len(max_connected_gr_amazon_movies.edges)))
print('AFTER: # of nodes: {} ... # of edges: {}'.format(len(grp_giant.nodes), len(grp_giant.edges)))

BEFORE: # of nodes: 8350 ... # of edges: 9253
AFTER: # of nodes: 8350 ... # of edges: 9253


In [10]:
# here is the check of the graph read
grp.Is_Connected_Bipartite(grp_giant)

Is connected: True ... Is bipartite: True


In [11]:
# this is to fetch all nodes without edges, so that we can use them to predict the edges
grp_giant_no_edges = grp.Create_Graph_From_List_NO_EDGE(grp_giant)
#if you check it, you'll see no edges but nodes
#grp_giant_no_edges.edges
#list(grp_giant_no_weights.nodes)[0:5]
#grp.Is_Connected_Bipartite(grp_giant_no_weights)

In [12]:
def Normalize_Array(arr_to_normalize, axis=0):
    if axis == 1:
        arr_sum_per_row = arr_to_normalize.sum(axis=1)
        arr_norm = arr_to_normalize/np.tile(arr_sum_per_row, (arr_sum_per_row.shape[0], 1)).T
    elif axis == 0:
        arr_sum_per_row = arr_to_normalize.sum(axis=0)
        arr_norm = arr_to_normalize/np.tile(arr_sum_per_row, (arr_sum_per_row.shape[1], 1)).T
    return arr_norm

# normalize P values (between 0 and 1) so that they can reflect probabilities 
# the probability with itself will be the highest
# !!! IMPORTANT !!!
# the probability with movie nodes will be ignored since we are looking for the relations(similarity-measure) 
# with other users
#P_norm = np.round(Normalize_Array(P), 6)

In [89]:
P = nx.to_numpy_matrix(max_connected_gr_amazon_movies)
P_norm = Normalize_Array(P)

#print(P)

# TEST - see if the matrix-P shows the correct values (to be sure if it's not changing the order)
# !!! IMPORTANT !!!
# apparently, it doesn't write the movies first and users last or doesn't follow any order..
node_0 = list(max_connected_gr_amazon_movies.nodes)[0]
node_1 = list(max_connected_gr_amazon_movies.nodes)[1]
edge_01 = max_connected_gr_amazon_movies.has_edge(node_0, node_1)
if edge_01 == True:
    weight_01 = max_connected_gr_amazon_movies.get_edge_data(node_0, node_1)['weight']

# Bxxxxxx = top_nodes = movies
print('node0={} .. node1={} .. edge01={} .. weight01={}'.format(node_0, node_1, edge_01, weight_01))
node_list = list(max_connected_gr_amazon_movies.nodes)
bottom_node_list = list(bottom_nodes)
top_node_list = list(top_nodes)
#print(node_list)
#print('\nbottom_nodes={}'.format(bottom_node_list))
print('\ntop_nodes={}'.format(top_node_list))

n_nodes = P_norm.shape[0]
# initiate R matrix which includes the r values of each node and
# having the highest relation of the node by itself, setting the diagonal to 1
R = np.diag(np.ones(n_nodes), 0)
beta = 0.15
#R *= beta
R_zero = R.copy()
#print(P)
#print(P_norm)
#print(R)
#print(R_zero)

node0=A141HP4LYPW .. node1=B003AI2VGA .. edge01=True .. weight01=3.0

top_nodes=['B00004RQBF', 'B00000IC8K', '6304888465', 'B0076XTH8K', '6305508569', 'B000VHVGMI', 'B0000764K0', 'B0001G6PZC', 'B0030FT96S', 'B004GIZX3Q', 'B002B900BM', 'B000063W82', 'B0051ZIXMQ', 'B000UGBOT0', 'B000KGGJ0Y', 'B001E6JCBC', '630403945X', 'B0002PYS4M', 'B001GBPZRU', 'B00000IC8M', 'B0002RQ2VW', 'B001LMU1A0', 'B0000544LY', '6304286961', 'B00151QYU8', 'B000Q7ZNZ4', 'B004BH1TN0', '6302643651', 'B00000IC87', 'B00004RQB1', '6301325842', 'B000NVL49W', '6304091338', '6303257933', 'B000CDSS86', 'B000065K9R', 'B00004CQTP', 'B00008WFTU', 'B000NVL49M', 'B000CNET20', 'B004SCMGSU', 'B000A9QK8M', 'B00000IC82', 'B004EPYZQ2', 'B0085KGGLY', 'B003BGZ60Y', 'B001I7HLTW', 'B000IOM0RY', '630436119X', 'B001AQT0VI', 'B000NDFLWG', 'B000KGGJ0E', '6304179499', '6300157423', 'B003F41T2Q', '6304040164', 'B0071AD95K', 'B000ANVPPG', 'B0018PH3K6', 'B000T4SWXO', '6305538409', '0790750708', 'B000FVQLQQ', 'B002OHDRF2', '1562229567', '63019012

In [91]:
# Bxxxxxx = top_nodes = movies
node_check = node_list[0]
if node_check in top_node_list:
    print('this is a movie: {}'.format(node_check))
else:
    print('this is a user: {}'.format(node_check))

this is a user: A141HP4LYPW


In [92]:
list(max_connected_gr_amazon_movies.edges([node_list[0]]))
#max_connected_gr_amazon_movies.edges(['B003AI2VGA']) 
#nx.edges(max_connected_gr_amazon_movies, ['B003AI2VGA'])
#node_list.index('B003AI2VGA')

[('A141HP4LYPW', 'B003AI2VGA'),
 ('A141HP4LYPW', 'B000063W1R'),
 ('A141HP4LYPW', '6304286961'),
 ('A141HP4LYPW', '5556167281'),
 ('A141HP4LYPW', '6303998690'),
 ('A141HP4LYPW', 'B000AMWIVM')]

In [93]:
user_indexes = []
movie_indexes = []
for nn in node_list:
    if nn in bottom_node_list:
        user_indexes.append(node_list.index(nn))
    else:
        movie_indexes.append(node_list.index(nn))

print(movie_indexes)
# users are usually too many, so will not print them
#print(user_indexes)

[1, 9, 34, 37, 47, 57, 365, 383, 394, 579, 581, 1111, 1117, 1128, 1130, 1132, 1520, 1558, 1585, 1603, 1629, 1633, 1705, 1713, 1747, 1777, 1954, 1956, 2097, 2100, 2230, 2234, 2238, 2822, 2827, 2830, 2835, 2852, 2879, 3020, 3095, 3101, 3241, 3273, 3288, 3300, 3450, 3513, 3523, 3947, 3973, 3975, 3980, 4008, 4052, 4085, 4086, 4088, 4095, 4099, 4115, 4387, 4391, 4403, 4406, 4414, 4422, 4425, 4466, 4508, 4510, 4520, 4565, 4566, 4567, 4610, 4670, 4672, 4774, 4813, 4840, 4993, 5027, 5267, 5269, 5273, 5280, 5284, 5349, 5388, 5464, 5540, 5560, 5571, 5592, 5599, 5763, 5771, 5790, 5798, 5809, 5825, 5847, 5875, 5899, 6045, 6047, 6059, 6111, 6118, 6122, 6155, 6175, 6200, 6207, 6240, 6244, 6254, 6259, 6266, 6274, 6279, 6290, 6294, 6303, 6370, 6383, 6391, 6656, 6675, 6712, 6719, 6722, 6728, 6734, 6746, 6761, 6805, 6817, 6891, 6892, 6903, 6907, 7047, 7098, 7111, 7113, 7151, 7153, 7220, 7236, 7509, 7525, 7921, 8075, 8078, 8100, 8110, 8128, 8129, 8131, 8145, 8204, 8231, 8233, 8238, 8270]


In [17]:
# SUM-check
#print('SUM-check')
#P_norm.sum(axis=1)

In [18]:
#savetxt(amp.FileNameUnique(prefix = "P_norm_", suffix = '.csv'), P_norm, delimiter=',')

In [19]:
#df_movies = amp.Reorganize_Edges_DataFrame(max_connected_gr_amazon_movies, input_type = 'Graph')

In [94]:
n_steps = 4

for i in range(n_steps):
    R = (1-beta)*np.dot(P_norm, R) + beta*R_zero
    print('step-{} completed .. date: {}'.format(i, datetime.datetime.now()))
    
#print(R)

step-0 completed .. date: 2020-05-12 14:43:33.578855
step-1 completed .. date: 2020-05-12 14:43:42.034392
step-2 completed .. date: 2020-05-12 14:43:50.141653
step-3 completed .. date: 2020-05-12 14:43:58.315293


In [99]:
# user similarity check
# fetch the values for node=0 (which is a user)
ref_user_idx = 2
# we will pick the most similar 10 users in this case
top_similarity = 10
user_test = np.array(list(R[ref_user_idx]))
print('R matrix values for the user (r vector of the user): \n{}'.format(user_test))

# sort the values of each column from smaller to the bigger
sorted_nodes = np.argsort(user_test)

user_similarity = []

# we will not consider the user himself or movies as similar nodes
for idx in sorted_nodes[0][0]:
    if idx != ref_user_idx and idx not in movie_indexes:
        user_similarity.append(idx)
        
user_similarity_top = user_similarity[-top_similarity:]

# ALL is too many, so we will not print it
#print('Ordered similarities-ALL: {}'.format(user_similarity))
print('Top similarities: {}'.format(user_similarity_top))

#for usr in user_similarity_top:
#    print(list(max_connected_gr_amazon_movies.nodes)[user_similarity_top[usr]])
#print(list(max_connected_gr_amazon_movies.nodes)[user_similarity_top[0]])


R matrix values for the user (r vector of the user): 
[[[2.18064548e-04 9.23411184e-03 8.50471378e-01 ... 1.33747041e-09
   1.78329388e-09 2.22911735e-09]]]
Top similarities: [3277, 3279, 3280, 3284, 3285, 547, 5, 4, 0, 3]


In [100]:
# top_nodes = MOVIES, bottom_nodes = users
user_reviewed = []
user_reviewed_id = []
movie_list_ref = []
movie_reviewed = []
movie_reviewed_id = []
is_reviewed = []
movie_score = []
review_count_list = []

movie_by_reference_user = list(max_connected_gr_amazon_movies.edges([node_list[ref_user_idx]]))

for edge_m in movie_by_reference_user:
    # if the second item is a user then we will add the first item to the movie_list
    if edge_m[1].startswith('A'):
        movie_list_ref.append(edge_m[0])
    else:
        movie_list_ref.append(edge_m[1])

for u_idx in user_similarity_top:
    user_name = node_list[u_idx]
    for m in movie_list_ref:
        m_idx = node_list.index(m)
        is_reviewed.append(max_connected_gr_amazon_movies.has_edge(user_name, m))
        user_reviewed.append(user_name)
        user_reviewed_id.append(u_idx)
        movie_reviewed.append(m)
        movie_reviewed_id.append(m_idx)
        try:
            scr = max_connected_gr_amazon_movies.get_edge_data(user_name, m)['weight']
        except:
            scr = 'NA'
        #scr = str(max_connected_gr_amazon_movies.get_edge_data(node_list[u_idx], m)['weight'])
        #max_connected_gr_amazon_movies.get_edge_data(node_list[u_idx], 'B003AI2VGA')['weight']
        movie_score.append(scr)
        review_count = len(list(max_connected_gr_amazon_movies.edges([m])))
        review_count_list.append(review_count)
        
        #is_reviewed = max_connected_gr_amazon_movies.get_edge_data(u, m)['weight']


dict_summary = {'user': user_reviewed, 'movie': movie_reviewed, 
                'user_id': user_reviewed_id, 'movie_id': movie_reviewed_id, 
                'is_reviewed': is_reviewed, 'score': movie_score,
               'n_review': review_count_list}

'''
dict_summary = {'user': user_reviewed, 'movie': movie_reviewed, 
                'user_id': user_reviewed_id, 'movie_id': movie_reviewed_id, 
                'is_reviewed': is_reviewed}
'''

df_summary = pd.DataFrame(dict_summary)

In [97]:
print(movie_by_reference_user)

[('A328S9RN3U5', 'B003AI2VGA'), ('A328S9RN3U5', 'B000063W1R'), ('A328S9RN3U5', 'B0001G6PZC'), ('A328S9RN3U5', 'B00022VM5I'), ('A328S9RN3U5', 'B001MBE63I'), ('A328S9RN3U5', 'B004EPYZQ2'), ('A328S9RN3U5', 'B000Q7ZNZ4'), ('A328S9RN3U5', 'B004EPYZQM'), ('A328S9RN3U5', 'B001OKUREO'), ('A328S9RN3U5', 'B000AMWIVM')]


In [101]:
true_values=len(df_summary.query("movie=='B003AI2VGA' and is_reviewed==True"))
total_values=len(df_summary.query("movie=='B003AI2VGA'"))

# review ratio by similar users
ratio_similar = true_values/total_values*100
print(ratio_similar)

40.0


In [102]:
ratio_review_list = []
review_count_list2 = []

for m in movie_list_ref:
    true_values = len(df_summary.query("movie=='" + m + "' and is_reviewed==True"))
    total_values = len(df_summary.query("movie=='" + m + "'"))
    review_count = len(list(max_connected_gr_amazon_movies.edges([m])))
    # review ratio by similar users
    ratio_similar = true_values/total_values*100
    ratio_review_list.append(ratio_similar)
    review_count_list2.append(review_count)

#dict_summary_ratio = {'movie': movie_list_ref, 'ratio_similar': ratio_review_list}
dict_summary_ratio = {'user-movie': movie_by_reference_user, 'ratio_similar': ratio_review_list,
                     'review_count': review_count_list2}

df_summary_ratio = pd.DataFrame(dict_summary_ratio)
n_reviews = len(max_connected_gr_amazon_movies.edges)
n_users = len(bottom_nodes)
n_movies = len(top_nodes)
print('Total # of MOVIEs: {} USERS: {} REVIEWs: {},  in the graph'.format(n_movies, n_users, n_reviews))
print(df_summary_ratio)

Total # of MOVIEs: 167 USERS: 8183 REVIEWs: 9253,  in the graph
                  user-movie  ratio_similar  review_count
0  (A328S9RN3U5, B003AI2VGA)           40.0             7
1  (A328S9RN3U5, B000063W1R)           10.0           309
2  (A328S9RN3U5, B0001G6PZC)            0.0           539
3  (A328S9RN3U5, B00022VM5I)            0.0            98
4  (A328S9RN3U5, B001MBE63I)           60.0            17
5  (A328S9RN3U5, B004EPYZQ2)            0.0           467
6  (A328S9RN3U5, B000Q7ZNZ4)            0.0            48
7  (A328S9RN3U5, B004EPYZQM)            0.0           273
8  (A328S9RN3U5, B001OKUREO)            0.0           199
9  (A328S9RN3U5, B000AMWIVM)           10.0           492


In [38]:
len(list(max_connected_gr_amazon_movies.edges(['B003AI2VGA'])))

7

In [31]:
df_summary.query("movie=='B000063W1R' and is_reviewed==True")

,user,movie,user_id,movie_id,is_reviewed
7,A2NJO6YE954,B000063W1R,318,57,True
49,A328S9RN3U5,B000063W1R,2,57,True


In [104]:
#pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
print(df_summary)

           user       movie  user_id  movie_id  is_reviewed score  n_review
0   AJOKH0DVQHA  B003AI2VGA     3277         1        False    NA         7
1   AJOKH0DVQHA  B000063W1R     3277        57        False    NA       309
2   AJOKH0DVQHA  B0001G6PZC     3277       581        False    NA       539
3   AJOKH0DVQHA  B00022VM5I     3277      3020        False    NA        98
4   AJOKH0DVQHA  B001MBE63I     3277      3273         True   5.0        17
5   AJOKH0DVQHA  B004EPYZQ2     3277      3523        False    NA       467
6   AJOKH0DVQHA  B000Q7ZNZ4     3277      4466        False    NA        48
7   AJOKH0DVQHA  B004EPYZQM     3277      5027        False    NA       273
8   AJOKH0DVQHA  B001OKUREO     3277      6907        False    NA       199
9   AJOKH0DVQHA  B000AMWIVM     3277      7525        False    NA       492
10  A3JMB1BG9WH  B003AI2VGA     3279         1        False    NA         7
11  A3JMB1BG9WH  B000063W1R     3279        57        False    NA       309
12  A3JMB1BG

In [48]:
## TEST
u_idx = 0
#weight_tst = max_connected_gr_amazon_movies.get_edge_data(node_list[u_idx], 'A141HP4LYPW')['weight']
weight_tst = max_connected_gr_amazon_movies.get_edge_data(node_list[u_idx], 'B003AI2VGA')['weight']
print(weight_tst)
list(max_connected_gr_amazon_movies.edges([node_list[u_idx]]))
print(node_list[9])

3.0
B00004CQT3


In [46]:
type(max_connected_gr_amazon_movies.get_edge_data(node_list[u_idx], 'B003AI2VGA'))

NoneType

In [47]:
type(max_connected_gr_amazon_movies)

networkx.classes.graph.Graph

In [ ]:
u_idx = 9
m='B003AI2VGA'
#max_connected_gr_amazon_movies.get_edge_data(node_list[u_idx], 'A141HP4LYPW')['weight']
type(max_connected_gr_amazon_movies.get_edge_data(node_list[u_idx], m)['weight'])